# 코드 설계

In [1]:
# 다시한번 생각해 볼 문제... 정호야 정신차리자
# 각각의 읽어온 파일을 sheet로 만들어 하나의 csv 파일로 저장할것. 왜 Why?
# 파일명 예시 nowon.csv, jongno.csv

# 과거의 논리구조 정리
# 데이터 이상치 처리 code
    # -9999, -999를 어떠한 값으로 대체해야하는 것에 대한 problem
    # 결측치를 어떻게 처리할 것인가? -> 무슨 값을 입력해 줄 것인가?
        # 1. 그대로 둔다. (100% 분석할 때 이상치로 될 것)
        # 2. None값으로 바꾼다. (연산할때 문제가 되는지 안되는지 확인해 볼 것)
        # 3. Excel에 있는 Fillter로 적합한 값을 찾아낸다. (빈 결측값들이 너무 많음)
        # 4. ... 그 외의 방법들 활용

## Data 가져오기 (노원구)

In [32]:
# import moduel
import pandas as pd
import numpy as np
import os 
from os import listdir
import re
import time

start = time.time()
# 데이터 읽기
base_dir = os.getcwd() # 현재 작업 dir 확인

### 각각 구로 작업할 것 (종로구, 노원구) ###
# 데이터가 있는 경로를 os 모듈을 활용해서 지정해 줄 것
# 후에 data_dir은 함수화를 시킬때, 인자값으로 들어갈 것이다.
data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구" # 데이터가 들어있는 폴더
working_dir = os.chdir(data_dir) # data_dir로 작업하는 폴더 경로 변경
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리\\"

files = os.listdir(data_dir) # data_dir의 파일 목록 추출
print("file_list: {}".format(files))
print("작업을 시작합니다.")

Merged_DF = pd.DataFrame(columns=['측정기 고유번호', '실외 측정기 구분', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)','초미세먼지(PM-2.5)'])

for file in files:
    raw_file = pd.read_csv(file, index_col=0)
    raw_file = raw_file.loc[:, ~raw_file.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제
    
    # 전부 결측치인 columns 제거
    raw_file = raw_file.drop(['co2','vocs'], axis='columns')
    
    # 데이터 결측치 처리 code -> Not a Number로 변경
    raw_file = raw_file.replace([-999,-9999],[np.nan, np.nan])
    
    # 데이터 이상치 처리 code -> 바로 앞의 값으로 대체
    # -> 작성해야함
    
    # column 이름 변경 code
    raw_file.rename(columns={
        'serial': '측정기 고유번호',
        'flag': '실외 측정기 구분',
        'pm10': '미세먼지(PM-10)',
#         'co2': '이산화탄소 농도(ppm)',
#         'vocs': '휘발성유기화합물 농도',
        'noise': '소음(db)',
        'temp': '온도(℃)',
        'humi': '습도(%)',
        'pm25': '초미세먼지(PM-2.5)',
    }, inplace=True)
    
    # date type 변경 code
    raw_file.index = pd.to_datetime(raw_file.index, format='%Y%m%d%H%M')
    
#     print(raw_file.columns) # columns name 확인
#     print(raw_file.describe())
#     print(raw_file.tail())
    
    # 바뀐 dataframe을 새로운 폴더의 같은 이름으로 to_csv 하는 code
    raw_file.to_csv(out_dir+file)
    print(file,"작업완료")
    
    # 하나의 DataFrame으로 연결
    Merged_DF = pd.concat([Merged_DF, raw_file])

# 합쳐진 df을 다른 이름으로 빼내기
Merged_DF.to_csv(out_dir+'MergedDF.csv')
print("working time: ", time.time() - start)

file_list: ['V10O1610102.csv', 'V10O1610200.csv', 'V10O1610293.csv', 'V10O1610297.csv', 'V10O1610312.csv', 'V10O1610351.csv', 'V10O1610356.csv', 'V10O1610376.csv', 'V10O1610610.csv', 'V10O1610616.csv', 'V10O1610629.csv', 'V10O1610630.csv', 'V10O1610642.csv', 'V10O1610643.csv', 'V10O1611097.csv', 'V10O1611100.csv', 'V10O1611102.csv', 'V10O1611104.csv', 'V10O1611150.csv', 'V10O1611229.csv', 'V10O1611652.csv', 'V10O1612126.csv']
작업을 시작합니다.
V10O1610102.csv 작업완료
V10O1610200.csv 작업완료
V10O1610293.csv 작업완료
V10O1610297.csv 작업완료
V10O1610312.csv 작업완료
V10O1610351.csv 작업완료
V10O1610356.csv 작업완료
V10O1610376.csv 작업완료
V10O1610610.csv 작업완료
V10O1610616.csv 작업완료
V10O1610629.csv 작업완료
V10O1610630.csv 작업완료
V10O1610642.csv 작업완료
V10O1610643.csv 작업완료


ValueError: time data 201804000000 does not match format '%Y%m%d%H%M' (match)

### Data 불러오기 (종로구)

## 데이터 EDA를 위한 코드 작성

## merge 데이터 확인

In [18]:
data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구_전처리" # 데이터가 들어있는 폴더
working_dir = os.chdir(data_dir)

TEMP = pd.read_csv('MergedDF_노원구.csv', index_col=0)

TEMP = round(TEMP.groupby(TEMP['측정기 고유번호']).mean(),2)

In [19]:
TEMP

,미세먼지(PM-10),소음(db),습도(%),실외 측정기 구분,온도(℃),초미세먼지(PM-2.5)
측정기 고유번호,,,,,,
V10O1610102,52.88,49.91,54.09,1.0,13.45,32.03
V10O1610200,56.19,48.98,51.50,1.0,12.55,33.96
V10O1610293,55.34,49.10,50.90,1.0,14.46,32.83
V10O1610297,48.99,35.13,54.41,1.0,18.56,25.56
V10O1610312,60.45,53.52,45.53,1.0,7.40,36.42
V10O1610351,61.34,53.15,47.57,1.0,5.60,37.38
V10O1610356,53.22,50.25,47.30,1.0,7.26,31.65
V10O1610376,55.24,53.76,51.85,1.0,14.23,32.32
V10O1610610,37.53,36.12,54.14,1.0,20.09,21.78


In [183]:
# df[:60]['미세먼지(PM-10)'].mean()

In [184]:
# df[:60]['초미세먼지(PM-2.5)'].mean()

In [96]:
# df.sort_index() # 이런 방식으로 인덱스를 기준으로 날짜 정렬이 가능하다. 일단 skip

In [227]:
# # 빈 데이터 프레임 생성 (컬럼 이름들을 가지고 있음) (+ 제일 위에)
# blank = pd.DataFrame(columns=['날짜', '측정기 고유번호', '미세먼지(PM-10)', '초미세먼지(PM-2.5)', '소음(db)', '온도(℃)', '습도(%)'])

# # 60행씩 넘어가는 for문 생성
# # for i in range(10000, len(df), 60): # test용
# # for i in range(0, len(df), 60): # 8580960
# # print(i, i+60)


# # pm10과 pm2.5의 평균을 내고, 각각 임시로 사용할 변수에 추가 (모조리 평균 내서 임시 변수로 만들기)
# pm10 = round(df[i:i+60]['미세먼지(PM-10)'].mean(),2)
# pm25 = round(df[i:i+60]['초미세먼지(PM-2.5)'].mean(),2)
# noise = round(df[i:i+60]['소음(db)'].mean(),2)
# wet = round(df[i:i+60]['습도(%)'].mean(),2)
# warm = round(df[i:i+60]['온도(℃)'].mean(),2)
# idx = df.index[i][:16]
# unique = df['측정기 고유번호'][i]

# # pm10 labeling
# if pm10 is None:
#     label10 = None
# elif pm10 <= 15:
#     label10 = '최고'
# elif pm10 >= 16 and pm10 <= 30:
#     label10 = '좋음'
# elif pm10 >= 31 and pm10 <= 40:
#     label10 = '양호'
# elif pm10 >= 41 and pm10 <= 50:
#     label10 = '보통'
# elif pm10 >= 51 and pm10 <= 75:
#     label10 = '나쁨'
# elif pm10 >= 76 and pm10 <= 100:
#     label10 = '상당히 나쁨'
# elif pm10 >= 101 and pm10 <= 150:
#     label10 = '매우 나쁨'
# else:
#     label10 = '최악'

# # pm25 labeling
# if pm25 is None:
#     label25 = None
# elif pm25 <= 8:
#     label25 = '최고'
# elif pm25 >= 9 and pm25 <= 15:
#     label25 = '좋음'
# elif pm25 >= 16 and pm25 <= 20:
#     label25 = '양호'
# elif pm25 >= 21 and pm25 <= 25:
#     label25 = '보통'
# elif pm25 >= 26 and pm25 <= 37:
#     label25 = '나쁨'
# elif pm25 >= 38 and pm25 <= 50:
#     label25 = '상당히 나쁨'
# elif pm25 >= 51 and pm25 <= 75:
#     label25 = '매우 나쁨'
# else:
#     label25 = '최악'

# # pd.Series 생성 (순서 중요합니다!, 데이터 숫자와 index 숫자 동일해야 합니다!)
# temp_S = pd.Series([idx, unique, noise, warm, wet, pm10, pm25, label10, label25],
#                    index=['날짜', '측정기 고유번호', '소음(db)', '온도(℃)', '습도(%)', '미세먼지(PM-10)', '초미세먼지(PM-2.5)', 'pm10단계', 'pm25단계'])

# # 빈 데이터 프레임에 추가
# blank = blank.append(temp_S, ignore_index=True)
# print(idx)
# print(unique,"작업 완료")

# # datetime으로 형식 변경 후, 날짜를 index로 바꿔줄 것
# # 굳이 할 필요 없을 것 같습니다.

# # csv로 저장
# out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리\\"
# blank.to_csv(out_dir+'노원구_환경기상_hour.csv')


## merged df 가지고 연습

In [228]:
# data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구_전처리" # 데이터가 들어있는 폴더
# working_dir = os.chdir(data_dir)

# test_df = pd.read_csv('MergedDF_노원구.csv')
# test_df.head()

In [229]:
# test_df.columns

## 시간단위 환경기상 데이터 프레임 만들기

In [40]:
data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구_전처리" # 데이터가 들어있는 폴더
working_dir = os.chdir(data_dir)

test_df = pd.read_csv('MergedDF_노원구.csv')

date_ls = list(test_df['Unnamed: 0'])

hour_ls = []
for date in date_ls:
    date = date[:13]
    hour_ls.append(date)

test_df['Unnamed: 0'] = hour_ls
test_df = test_df.rename(columns={'Unnamed: 0': '날짜'})

hour_df = round(test_df.groupby(test_df['날짜']).mean(),2)
hour_df = hour_df.drop(columns=['실외 측정기 구분'])

In [48]:
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'

soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'

bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'

terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'

soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'

bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'

terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

# for i in range(len(hour_df)):
#     x = hour_df['초미세먼지(PM-2.5)'][i]
    
#     if x <= 15.0:
#         hour_df['PM25단계'][i] = '좋음'
#     elif x >= 16.0 and x <= 35.0:
#         hour_df['PM25단계'][i] = '보통'
#     elif x >= 36.0 and x <= 75.0:
#         hour_df['PM25단계'][i] = '나쁨'
#     elif x >= 76.0:
#         hour_df['PM25단계'][i] = '매우나쁨'

# for i in range(len(hour_df)):
#     x = hour_df['미세먼지(PM-10)'][i]
    
#     if x <= 30.0:
#         hour_df['PM10단계'][i] = '좋음'
#     elif x >= 31.0 and x <= 80.0:
#         hour_df['PM10단계'][i] = '보통'
#     elif x >= 81.0 and x <= 150.0:
#         hour_df['PM10단계'][i] = '나쁨'
#     elif x >= 151.0:
#         hour_df['PM10단계'][i] = '매우나쁨'

hour_df

,미세먼지(PM-10),소음(db),습도(%),온도(℃),초미세먼지(PM-2.5),PM10단계,PM25단계
날짜,,,,,,,
2018-04-01 00,52.82,48.41,52.48,14.22,29.64,보통,보통
2018-04-01 01,55.17,48.25,53.32,13.67,30.13,보통,보통
2018-04-01 02,57.04,47.87,52.78,13.58,28.51,보통,보통
2018-04-01 03,53.81,47.71,52.59,13.36,26.91,보통,보통
2018-04-01 04,51.53,47.71,53.20,13.31,28.96,보통,보통
2018-04-01 05,54.25,47.71,53.61,13.38,29.40,보통,보통
2018-04-01 06,58.34,48.04,52.77,13.64,31.79,보통,보통
2018-04-01 07,59.03,48.16,52.31,14.01,32.95,보통,보통
2018-04-01 08,64.59,48.70,53.83,14.44,36.28,보통,나쁨


In [49]:
hour_df['PM25단계'].value_counts()

보통       3961
좋음       2281
나쁨       2090
매우 나쁨     428
Name: PM25단계, dtype: int64

In [50]:
hour_df['PM10단계'].value_counts()

보통      5471
좋음      1990
나쁨      1079
매우나쁨     220
Name: PM10단계, dtype: int64

In [53]:
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리\\"

hour_df.to_csv(out_dir+'노원구_hour.csv')

In [54]:
pd.read_csv('노원구_hour.csv')

,날짜,미세먼지(PM-10),소음(db),습도(%),온도(℃),초미세먼지(PM-2.5),PM10단계,PM25단계
0,2018-04-01 00,52.82,48.41,52.48,14.22,29.64,보통,보통
1,2018-04-01 01,55.17,48.25,53.32,13.67,30.13,보통,보통
2,2018-04-01 02,57.04,47.87,52.78,13.58,28.51,보통,보통
3,2018-04-01 03,53.81,47.71,52.59,13.36,26.91,보통,보통
4,2018-04-01 04,51.53,47.71,53.20,13.31,28.96,보통,보통
5,2018-04-01 05,54.25,47.71,53.61,13.38,29.40,보통,보통
6,2018-04-01 06,58.34,48.04,52.77,13.64,31.79,보통,보통
7,2018-04-01 07,59.03,48.16,52.31,14.01,32.95,보통,보통
8,2018-04-01 08,64.59,48.70,53.83,14.44,36.28,보통,나쁨
9,2018-04-01 09,70.49,48.96,54.67,15.38,38.05,보통,나쁨


## 일별 단위 데이터 프레임 만들기

In [55]:
data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구_전처리" # 데이터가 들어있는 폴더
working_dir = os.chdir(data_dir)

test_df = pd.read_csv('MergedDF_노원구.csv')
test_df.head()

date_ls = list(test_df['Unnamed: 0'])

days_ls = []
for date in date_ls:
    date = date[:10]
    days_ls.append(date)
    
test_df['Unnamed: 0'] = days_ls
test_df = test_df.rename(columns={'Unnamed: 0': '날짜'})

days_df = round(test_df.groupby(test_df['날짜']).mean(),2)
days_df = days_df.drop(columns=['실외 측정기 구분'])

In [56]:
days_df['PM10단계'] = None
days_df['PM25단계'] = None

good = days_df[days_df['미세먼지(PM-10)'] < 31.0].index
days_df['PM10단계'].loc[good] = '좋음'

soso = days_df[days_df['미세먼지(PM-10)'] >= 31.0].index
days_df['PM10단계'].loc[soso] = '보통'

bad = days_df[days_df['미세먼지(PM-10)'] >= 81.0].index
days_df['PM10단계'].loc[bad] = '나쁨'

terrible = days_df[days_df['미세먼지(PM-10)'] >= 151.0].index
days_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = days_df[days_df['초미세먼지(PM-2.5)'] < 16.0].index
days_df['PM25단계'].loc[good] = '좋음'

soso = days_df[days_df['초미세먼지(PM-2.5)'] >= 16.0].index
days_df['PM25단계'].loc[soso] = '보통'

bad = days_df[days_df['초미세먼지(PM-2.5)'] >= 36.0].index
days_df['PM25단계'].loc[bad] = '나쁨'

terrible = days_df[days_df['초미세먼지(PM-2.5)'] >= 76.0].index
days_df['PM25단계'].loc[terrible] = '매우 나쁨'

In [57]:
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리\\"

days_df.to_csv(out_dir+'노원구_days.csv')

In [37]:
# days_df['PM10단계'] = None

# for i in range(len(days_df)):
#     x = days_df['미세먼지(PM-10)'][i]
    
#     if x <= 30.0:
#         days_df['PM10단계'][i] = '좋음'
#     elif x >= 31.0 and x <= 80.0:
#         days_df['PM10단계'][i] = '보통'
#     elif x >= 81.0 and x <= 150.0:
#         days_df['PM10단계'][i] = '나쁨'
#     elif x >= 151.0:
#         days_df['PM10단계'][i] = '매우나쁨'

# days_df

In [34]:
# days_df['PM10단계'].value_counts()

In [36]:
# days_df['PM25단계'] = None

# for i in range(len(days_df)):
#     x = days_df['초미세먼지(PM-2.5)'][i]
    
#     if x <= 15.0:
#         days_df['PM25단계'][i] = '좋음'
#     elif x >= 16.0 and x <= 35.0:
#         days_df['PM25단계'][i] = '보통'
#     elif x >= 36.0 and x <= 75.0:
#         days_df['PM25단계'][i] = '나쁨'
#     elif x >= 76.0:
#         days_df['PM25단계'][i] = '매우나쁨'

# days_df

In [35]:
# days_df['PM25단계'].value_counts()

## 월별 단위 데이터 프레임 만들기

In [3]:
import os
import numpy as np
import pandas as pd

data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구_전처리" # 데이터가 들어있는 폴더
working_dir = os.chdir(data_dir)

test_df = pd.read_csv('MergedDF_노원구.csv')

date_ls = list(test_df['Unnamed: 0'])

months_ls = []
for date in date_ls:
    date = date[:7]
    months_ls.append(date)

test_df['Unnamed: 0'] = months_ls
test_df = test_df.rename(columns={'Unnamed: 0': '날짜'})

months_df = round(test_df.groupby(test_df['날짜']).mean(),2)
months_df = months_df.drop(columns=['실외 측정기 구분'])

In [4]:
# months_df['PM10단계'] = None
# months_df['PM25단계'] = None

# for i in range(len(months_df)):
#     if months_df['미세먼지(PM-10)'][i] <= 15.0:
#         months_df['PM10단계'][i] = '최고'
#     elif months_df['미세먼지(PM-10)'][i] >= 50.0:
#         months_df['PM10단계'][i] = '최악'
#     else:
#         months_df['PM10단계'][i] = '평범'
months_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,미세먼지(PM-10),소음(db),습도(%),온도(℃),초미세먼지(PM-2.5),PM10단계,PM25단계
날짜,,,,,,,
2018-04,51.12,48.89,48.61,14.46,25.89,최악,None
2018-05,48.38,48.91,57.23,19.68,25.91,평범,None
2018-06,40.61,48.84,55.70,24.62,24.56,평범,None
2018-07,34.87,50.34,62.79,29.01,19.36,평범,None
2018-08,36.98,50.50,60.65,29.92,20.07,평범,None
2018-09,34.10,50.31,57.49,22.94,14.54,평범,None
2018-10,42.66,52.16,54.96,13.40,23.57,평범,None
2018-11,61.81,52.48,54.08,8.95,35.73,최악,None
2018-12,52.85,52.74,40.88,0.52,30.25,최악,None


In [ ]:
    # labeling code
    # 이 코드 수정해야합니다
#     if raw_file['pm10'] is not None:
#         excellent = raw_file.loc[raw_file['pm10']<=15,:].index
#         raw_file['pm10_label'][excellent] = '최고'
        
#         good = raw_file.loc[raw_file['pm10']>=16,:].index
#         raw_file['pm10_label'][good] = '좋음'
        
#         yangho = raw_file.loc[raw_file['pm10']>=31,:].index
#         raw_file['pm10_label'][yangho] = '양호'
        
#         soso = raw_file.loc[raw_file['pm10']>=41,:].index
#         raw_file['pm10_label'][soso] = '보통'
        
#         bad = raw_file.loc[raw_file['pm10']>=51,:].index
#         raw_file['pm10_label'][bad] = '나쁨'
        
#         very_bad = raw_file.loc[raw_file['pm10']>=76,:].index
#         raw_file['pm10_label'][very_bad] = '상당히 나쁨'
        
#         terrible = raw_file.loc[raw_file['pm10']>=101,:].index
#         raw_file['pm10_label'][terrible] = '매우 나쁨'
        
#         worst = raw_file.loc[raw_file['pm10']>=151,:].index
#         raw_file['pm10_label'][worst] = '최악'
        
#     if raw_file['pm25'] is not None:
#         excellent = raw_file.loc[raw_file['pm25']<=8,:].index
#         raw_file['pm25_label'][excellent] = '최고'
        
#         good = raw_file.loc[raw_file['pm25']>=9,:].index
#         raw_file['pm25_label'][good] = '좋음'
        
#         yangho = raw_file.loc[raw_file['pm25']>=16,:].index
#         raw_file['pm25_label'][yangho] = '양호'
        
#         soso = raw_file.loc[raw_file['pm25']>=21,:].index
#         raw_file['pm25_label'][soso] = '보통'
        
#         bad = raw_file.loc[raw_file['pm25']>=26,:].index
#         raw_file['pm25_label'][bad] = '나쁨'
        
#         very_bad = raw_file.loc[raw_file['pm25']>=38,:].index
#         raw_file['pm25_label'][very_bad] = '상당히 나쁨'
        
#         terrible = raw_file.loc[raw_file['pm25']>=51,:].index
#         raw_file['pm25_label'][terrible] = '매우 나쁨'
        
#         worst = raw_file.loc[raw_file['pm25']>=76,:].index
#         raw_file['pm25_label'][worst] = '최악'
    
#     print("\n")
#     print("="*25,file,"="*25)
#     print(type(raw_file)) # DataFrame 형태